<a href="https://colab.research.google.com/github/agemagician/ProtTrans/blob/master/Embedding/TensorFlow/Advanced/ProtT5-XL-UniRef50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Important Notes:
1. ProtT5-XL-UniRef50 has both encoder and decoder, for feature extraction we only load and use the encoder part.
2. Loading only the encoder part, reduces the inference speed and the GPU memory requirements by half.
3. In order to use ProtT5-XL-UniRef50 encoder, you must install the latest huggingface transformers version from their GitHub repo.
4. If you are intersted in both the encoder and decoder, you should use TFT5Model rather than TFT5EncoderModel.

<h3>Extracting protein sequences' features using ProtT5-XL-UniRef50 pretrained-model</h3>


**1. Load necessry libraries including huggingface transformers**

In [1]:
!pip install -q SentencePiece transformers

     |████████████████████████████████| 1.2MB 7.6MB/s 
     |████████████████████████████████| 2.1MB 53.7MB/s 
     |████████████████████████████████| 3.2MB 55.0MB/s 
     |████████████████████████████████| 890kB 59.1MB/s 


In [2]:
from transformers import TFT5EncoderModel, T5Tokenizer
import numpy as np
import re
import gc

<b>2. Load the vocabulary and ProtT5-XL-UniRef50 Model<b>

In [3]:
tokenizer = T5Tokenizer.from_pretrained("Rostlab/prot_t5_xl_uniref50", do_lower_case=False )

In [4]:
model = TFT5EncoderModel.from_pretrained("Rostlab/prot_t5_xl_uniref50", from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFT5EncoderModel: ['decoder.block.12.layer.0.layer_norm.weight', 'decoder.block.12.layer.1.EncDecAttention.o.weight', 'decoder.block.4.layer.0.SelfAttention.q.weight', 'decoder.block.13.layer.1.EncDecAttention.v.weight', 'decoder.block.14.layer.0.SelfAttention.q.weight', 'decoder.block.10.layer.1.EncDecAttention.q.weight', 'decoder.block.19.layer.1.layer_norm.weight', 'decoder.block.18.layer.2.DenseReluDense.wi.weight', 'decoder.block.23.layer.2.DenseReluDense.wo.weight', 'decoder.block.5.layer.1.EncDecAttention.q.weight', 'lm_head.weight', 'decoder.block.20.layer.0.SelfAttention.o.weight', 'decoder.block.2.layer.1.EncDecAttention.v.weight', 'decoder.block.4.layer.0.SelfAttention.v.weight', 'decoder.block.15.layer.0.SelfAttention.k.weight', 'decoder.block.17.layer.1.layer_norm.weight', 'decoder.block.2.layer.1.layer_norm.weight', 'decoder.block.13.layer.1.layer_norm.weight', 'decoder.block.16.layer.2.Den

In [5]:
gc.collect()

50

<b>3. Create or load sequences and map rarely occured amino acids (U,Z,O,B) to (X)<b>

In [6]:
sequences_Example = ["A E T C Z A O","S K T Z P"]

In [7]:
sequences_Example = [re.sub(r"[UZOB]", "X", sequence) for sequence in sequences_Example]

<b>4. Tokenize, encode sequences and load it into the GPU if possibile<b>

In [8]:
ids = tokenizer.batch_encode_plus(sequences_Example, add_special_tokens=True, padding=True, return_tensors="tf")

In [9]:
input_ids = ids['input_ids']
attention_mask = ids['attention_mask']

<b>5. Extracting sequences' features and load it into the CPU if needed<b>

In [10]:
embedding = model(input_ids)

In [11]:
embedding = np.asarray(embedding.last_hidden_state)

In [12]:
attention_mask = np.asarray(attention_mask)

<b>7. Remove padding (\<pad\>) and special tokens (\</s\>) that is added by ProtT5-XL-UniRef50 model<b>

In [13]:
features = [] 
for seq_num in range(len(embedding)):
    seq_len = (attention_mask[seq_num] == 1).sum()
    seq_emd = embedding[seq_num][:seq_len-1]
    features.append(seq_emd)

In [14]:
print(features)

[array([[ 0.22006367, -0.10348108, -0.24565518, ...,  0.07009576,
         0.18895069, -0.12561187],
       [ 0.10839709, -0.14164549, -0.2611331 , ...,  0.0752494 ,
        -0.19057555,  0.14047642],
       [ 0.20587248, -0.06389292, -0.3473575 , ..., -0.025124  ,
         0.00397164,  0.00148447],
       ...,
       [ 0.4088016 , -0.08833068, -0.12417938, ...,  0.0790412 ,
         0.47919267,  0.16784595],
       [ 0.09277283, -0.08904049, -0.2286643 , ..., -0.12425306,
         0.15413284, -0.22367832],
       [ 0.3757485 , -0.05730629, -0.19935629, ...,  0.04669633,
         0.27533904, -0.10681018]], dtype=float32), array([[ 0.1954189 , -0.10268322, -0.23747759, ..., -0.14768864,
        -0.04564754, -0.00130377],
       [ 0.15905248, -0.12580584, -0.04593242, ..., -0.04285887,
        -0.2786491 ,  0.02392592],
       [ 0.335001  , -0.08294542, -0.08190978, ..., -0.02507642,
         0.08196809, -0.17845191],
       [ 0.23378026, -0.19225256, -0.1894767 , ...,  0.05166554,
     